# Лабораторная работа 3. Полиномиальная регрессия. Регуляризация

### Задание 1. Полиномиальная регрессия

Загрузите данные из файлов `ml_lab1_train.txt` и `ml_lab1_test.txt` (первая лабораторная работа).

Для $k = 1,2,3,\ldots,10$ найдите полином ${f}_k$ степени $k$, наилучшим образом приближающий неизвестную зависимость.

Выведите коэффициенты полиномов и нарисуйте их графики на одном чертеже вместе с точками данных $(x_i, y_i)$ (возможно, чертеж стоит сделать побольше; это делается командой `plt.figure(figsize=(width, height))`).

Для каждого из полиномов найдите среднеквадратическую ошибку $MSE$ и коэффициент детерминации $R^2$ на обучающих данных и на тестовых данных. Постройте графики зависимости $MSE$ на обучающих и тестовых данных в зависимости от степени полинома (должно быть три графика: один для обучающих данных, второй – для тестовых, третий – для обоих видов данных вместе).

Сделайте вывод, что происходит с ошибкой на обучающих и тестовых данных с увеличением степени полинома.

In [3]:
import numpy as np
import scipy.linalg as sla
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import statistics 
from sklearn.preprocessing import PolynomialFeatures
import random
%matplotlib inline

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
# Загрузка данных
data_train = np.loadtxt('ml_lab1_train.txt', delimiter=',')
data_test = np.loadtxt('ml_lab1_test.txt', delimiter=',')
# Разделение данных и целевых значений для обучающих данных
X_train = data_train[:,0]
y_train = data_train[:,1]
# Разделение данных и целевых значений для тестовых данных
X_test = data_test[:,0]
y_test = data_test[:,1]

### Полиномиальная регрессия на основе нормального уравнения

In [ ]:
# Функция нахождения значений по линейной функции с предсказанными весовыми коэффициентами
def f_reg(X, w):
    X = np.column_stack([np.ones(X.shape[0]).T, X])
    y_predict = np.dot(w, X.T)
    return y_predict

In [ ]:
# Функция нахождения вектора весов на основе нормального уравнения (a = 0 без регуляризации, а != 0 с регуляризацией
def get_weight(X, y, a=0):
    X = np.column_stack([np.ones(X.shape[0]).T, X])    
    w = np.dot(np.linalg.inv(np.dot(X.T, X) + a * np.eye(X.shape[1])), np.dot(X.T, y))
    return w  # Возвращаем вектор весов

In [ ]:
# Создание полиномиальных признаков
def create_PolynomialFeatures(degree, X_train, X_test):
    poly = PolynomialFeatures(degree, include_bias=False)
    X_poly_train = poly.fit_transform(X_train.reshape(-1,1))
    X_poly_test = poly.fit_transform(X_test.reshape(-1,1))
    return X_poly_train, X_poly_test

In [ ]:
# Полиномиальная регрессия на основе нормального уравнения
def poli_reg(X_train, X_test, y_train, degree):
    X_poly_train, X_poly_test = create_PolynomialFeatures(i + min_degree, X_train, X_test)    
    w = get_weight(X_poly_train, y_train)
    return X_poly_train, X_poly_test, w

In [ ]:
mse_train = []
r2_train = []
mse_test = []
r2_test = []
arr_w = []
arr_y_predict_train = []
arr_y_predict_test = []

print(X_train.shape[0])
count = 10
min_degree = 1
for i in range(count):
    X_poly_train, X_poly_test, w = poli_reg(X_train, X_test, y_train, i + min_degree)
    arr_w.append(w)
    
    y_predict_train = f_reg(X_poly_train, w)
    arr_y_predict_train.append(y_predict_train)
    y_predict_test = f_reg(X_poly_test, w)
    arr_y_predict_test.append(y_predict_test)
    
    mse_train.append(mean_squared_error(y_train, y_predict_train))
    r2_train.append(r2_score(y_train, y_predict_train))
    
    mse_test.append(mean_squared_error(y_test, y_predict_test))
    r2_test.append(r2_score(y_test, y_predict_test))
    print('Степень = ', i + min_degree, '\tw = ', w, '\n\nMSE_train = ', mse_train[i], '\tr2_train = ', r2_train[i], '\nMSE_test = ', mse_test[i], '\tr2_test = ', r2_test[i],'\n\n')

In [ ]:
# Задание 1. Полиномиальная регрессия (на основе нормального уравнения). Построение полученных полиномов

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
width = 15
height = 10
m = 0.1

plt.figure(figsize=(width, height))
plt.title('Полиномиальная регрессия')
plt.xlabel('x')
plt.ylabel('y')
plt.scatter(X_train, y_train, 40, 'g', 'o', alpha=0.8, label='train')
plt.scatter(X_test, y_test, 40, 'b', 'x', alpha=0.8, label='test')

for i in range(10):
    plt.plot(X_train, arr_y_predict_train[i], color = colors[i], label='Полином '+str(i + min_degree)+' степени')
    plt.legend(loc = 'best', prop = {'size': 10})

plt.show()

In [ ]:
# Задание 1. Построение графиков зависимости функции ошибки от степени полинома
width = 15
height = 5
m = 1
plt.figure(figsize=(width, height))
x = [i+1 for i in range(count)]
# 1-ый график - ошибка на обучающем наборе
sp = plt.subplot(131)
plt.plot(x, mse_train, color = 'green', label='train')
plt.title('MSE_train')
plt.xlabel('degree')
plt.ylabel('MSE')
# 2-ой график - ошибка на тестовом наборе
sp = plt.subplot(132)
plt.plot(x, mse_test, color = 'blue', label='test')
plt.title('MSE_test')
plt.xlabel('degree')
plt.ylabel('MSE')
# 3-ий график - графики ошибок на обучающем и тестовом наборах
sp = plt.subplot(133)
plt.plot(x, mse_train, color = 'green', label='train')
plt.plot(x, mse_test, color = 'blue', label='test')
plt.legend(loc = 'best', prop = {'size': 10})
plt.title('MSE_train & test')
plt.xlabel('degree')
plt.ylabel('MSE')
plt.show()

In [ ]:
best_degree = mse_test.index(min(mse_test)) + min_degree
print('Вывод: наилучший результат достигается при', best_degree, 'степени. При дальнейшем увеличении степени полинома происходит переобучение.')

### ***Выполнение задания 1 на основе градиентного спуска***

In [ ]:
# Вычисление градиента
def gradient(X, y, w):
    return 2 * np.dot(X.T, np.dot(X, w) - y) / y.shape[0]

In [ ]:
# Обобщенный вариант функции градиентного спуска

# По умолчанию (learn_rate=0.5, n_iter=100, eps=1e-06, batch_size=len(y), stochastic=False) - классический градиентный спуск
# При задании stochastic = True и batch_size < len(y) - стохастический по мини-батчам

def gradient_descent_all(X, y, w, learn_rate=0.5, n_iter=100, eps=1e-06, batch_size=None, stochastic=False):
    if stochastic:
        n_obs = X.shape[0] # Количество наблюдений (значений)
        xy = np.c_[X.reshape(n_obs, 1), y.reshape(n_obs, 1)] # Объединение массивов x и y в один
    
    k_iter = 0
    n = learn_rate
    for _ in range(n_iter):
        if stochastic:
            rng.shuffle(xy) # Перемешивание x и y
            start = np.random.randint(n_obs-batch_size) # Выбор случайного значения
            stop = start + batch_size
            x_batch, y_batch = np.array(xy[start:stop, :-1].ravel().tolist()), np.array(xy[start:stop, -1:].ravel().tolist()) # Формирование мини-батча и разделение x и y
        else:
            x_batch = X
            y_batch = y
        
        x_batch = np.column_stack([np.ones(x_batch.shape[0]).T, x_batch])            
        
        diff = learn_rate * gradient(x_batch, y_batch, w)
        w -= diff
        
        k_iter += 1    
        if np.all(np.abs(diff) <= eps):
            break     
        learn_rate -= n / n_iter
    return w, k_iter

In [ ]:
# Градиентный спуск
rng = np.random.default_rng()

mse_train = []
r2_train = []
mse_test = []
r2_test = []
arr_w = []

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
width = 10
height = 7
m = 0.1
plt.figure(figsize=(width, height))
x_min = min(X_train.min(), X_test.min())
x_max = max(X_train.max(), X_test.max())
y_min = min(y_train.min(), y_test.min())
y_max = max(y_train.max(), y_test.max())
plt.scatter(X_train, y_train, 40, 'g', 'o', alpha=0.8, label='train')
plt.scatter(X_test, y_test, 40, 'b', 'x', alpha=0.8, label='test')
plt.xlim(x_min - m, x_max + m)
plt.ylim(y_min - m, y_max + m)
plt.legend(loc = 'best', prop = {'size': 10})

count = 10
min_degree = 1
for i in range(count):
    # Создание полиномиальных признаков
    poly = PolynomialFeatures(i + min_degree, include_bias=False)
    X_poly_train = poly.fit_transform(X_train.reshape(-1,1))
    X_poly_test = poly.fit_transform(X_test.reshape(-1,1))
    w_0 = np.ones(X_poly_train.shape[1] + 1)
    w, k = gradient_descent_all(X_poly_train, y_train, w_0, learn_rate=1, n_iter = 120)
    #w, k = gradient_descent(X_poly_train, y_train, w, learn_rate=0.5, n_iter=100, eps=1e-06, batch_size=len(y), stochastic=False)
    arr_w.append(w)
    y_predict_train = f_reg(X_poly_train, w)
    plt.plot(X_train, y_predict_train, color = colors[i], label=str(i + min_degree))
    plt.legend(loc = 'best', prop = {'size': 10})
    mse_train.append(mean_squared_error(y_train, y_predict_train))
    r2_train.append(r2_score(y_train, y_predict_train))
    y_predict_test = f_reg(X_poly_test, w)
    mse_test.append(mean_squared_error(y_test, y_predict_test))
    r2_test.append(r2_score(y_test, y_predict_test))
    print('Степень = ', i + min_degree, 'w = ', w, ' Количество итераций = ', k, '\nMSE_train = ', mse_train[i], 'r2_train = ', r2_train[i], '\nMSE_test = ', mse_test[i], 'r2_test = ', r2_test[i],'\n')
    #val_err(X_poly_train, y, mse_train, r2_train)

plt.title('Полиномиальная регрессия')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# Построение графиков функции ошибки
width = 10
height = 15
m = 1
plt.figure(figsize=(width, height))
x = [i+1 for i in range(count)]
#subplot train
sp = plt.subplot(311)
plt.plot(x, mse_train, color = 'green', label='train')
plt.title('MSE_train')
plt.xlabel('degree')
plt.ylabel('MSE')
#subplot test
sp = plt.subplot(312)
plt.plot(x, mse_test, color = 'blue', label='test')
plt.title('MSE_test')
plt.xlabel('degree')
plt.ylabel('MSE')
#subplot train and test
sp = plt.subplot(313)
plt.plot(x, mse_train, color = 'green', label='train')
plt.plot(x, mse_test, color = 'blue', label='test')
plt.legend(loc = 'best', prop = {'size': 10})
plt.title('MSE')
plt.xlabel('degree')
plt.ylabel('MSE')
plt.show()

In [ ]:
print('Вывод: наилучший результат достигается при', mse_test.index(min(mse_test)) + 1, 'степени. При дальнейшем увеличении степени полинома результат практически не меняется.')

### Задание 2. Регуляризация

Вспомним, что задачу линейной регрессии (нахождение вектора коэффициентов $\overrightarrow{w}_{opt}$) можно решить при помощи нормального уравнения:

$$\overrightarrow{w}_{opt} = \left(X^TX\right)^{-1}X^T\overrightarrow{y}.$$

Если строки или столбцы матрицы $X$ линейно зависимы, то матрица $X^TX$ вырожденна и задача не может быть решена с помощью нормального уравнения (придется использовать, например, градиентный спуск). На практике различные признаки редко бывают *в точности* линейно зависимы, однако во многих ситуациях они скоррелированы и становятся "почти" линейно зависимыми. Таковы, к примеру, зарплата человека, его уровень образования, цена машины и суммарная площадь недвижимости, которой он владеет. В этом случае матрица $X^TX$ будет близка к вырожденной, и это приводит к численной неустойчивости и плохому качеству решений; как следствие, будет иметь место *переобучение* (overfitting). Один из симптомов этой проблемы – необычно большие по модулю компоненты вектора $\overrightarrow{w}_{opt}$.

Один из способов борьбы с переобучением – **регуляризация**. Сейчас мы рассмотрим одну из её разновидностей – **L2-регуляризацию**. Идея в том, чтобы подправить матрицу $X^TX$, сделав её "получше". Например, это можно сделать, заменив её на $(X^TX + \alpha I)$, где $\alpha$ – параметр регуляризации, $I$ – единичная матрица. Пожертвовав точностью на обучающей выборке, мы тем не менее получаем численно более стабильное псевдорешение:

$$\overrightarrow{w}_{opt} = (X^TX + \alpha I)^{-1}X^T\overrightarrow{y}$$

и снижаем эффект переобучения. Параметр $\alpha$ нужно подбирать, и каких-то универсальных способов это делать нет, но зачастую можно его подобрать таким, чтобы ошибка на тестовой выборке падала. 

Теперь давайте вспомним первую задачу. Если вы её сделали, то помните, что ошибка аппроксимации полиномом шестой степени довольно высокая. Убедитесь, что, используя регуляризацию с хорошо подобранным параметром $\alpha$, ошибку на тестовой выборке можно сделать не больше, чем для полинома оптимальной степени в модели без регуляризации. Для этого $\alpha$ сравните $\det(X^TX)$ и $\det(X^TX + \alpha I)$.

Изобразите на графике три полинома: полином оптимальной степени без регуляризации, полином шестой степени без регуляризации и полином шестой степени с регуляризацией.

In [ ]:
# Выше описана функция нахождения вектора весов на основе нормального уравнения с L2-регуляризацией (a != 0)
# get_weight(X, y, a=0):
#    X = np.column_stack([np.ones(X.shape[0]).T, X])
#    w = np.dot(np.linalg.inv(np.dot(X.T, X) + a * np.eye(X.shape[1])), np.dot(X.T, y))
#    return w  # Возвращаем вектор весов

In [ ]:
alpha_values = [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.08, 1, 2, 3, 5, 8, 10, 20, 50, 100]
# Создание полиномиальных признаков
X_poly_train, X_poly_test = create_PolynomialFeatures(6, X_train, X_test)
mse_train = []
mse_test = []
arr_w = []
for a in alpha_values:
    w = get_weight(X_poly_train, y_train, a)
    arr_w.append(w)
    y_predict_train = f_reg(X_poly_train, w)
    mse_train.append(mean_squared_error(y_train, y_predict_train))
    y_predict_test = f_reg(X_poly_test, w)
    mse_test.append(mean_squared_error(y_test, y_predict_test))
    print('a =', a, '\tw = ', w, '\nMSE_train = ', mse_train[len(mse_train)-1], '\nMSE_test = ', mse_test[len(mse_train)-1],'\n')
print('Вывод: наилучший результат достигается при a =', alpha_values[mse_test.index(min(mse_test))])   

In [ ]:
# Сравним определители det(X.T*X) и det(X.T*X + alpha*I)
X = np.column_stack([np.ones(X_poly_train.shape[0]).T, X_poly_train])
print('det(X.T*X) =', np.linalg.det(np.dot(X.T, X)))
print('det(X.T*X + alpha*I) =', np.linalg.det(np.dot(X.T, X) + a * np.eye(X.shape[1])))

In [ ]:
X_poly_train, X_poly_test = create_PolynomialFeatures(6, X_train, X_test)

w6 = get_weight(X_poly_train, y_train, a = 0) # Полином 6-ой степени без регуляризации
w6_a = get_weight(X_poly_train, y_train, a = 1) # Полином 6-ой степени с регуляризацией (a = 1)
y6_predict_train = f_reg(X_poly_train, w6)
y6_a_predict_train = f_reg(X_poly_train, w6_a)

X_poly_train, X_poly_test = create_PolynomialFeatures(best_degree, X_train, X_test)
w_best = get_weight(X_poly_train, y_train, a = 0) # Полином 5-ей степени без регуляризации (лучшая степень полинома)
y_best_predict_train = f_reg(X_poly_train, w_best)

In [ ]:
# Построение графиков
width = 15
height = 10
m = 0.1
plt.figure(figsize=(width, height))
plt.scatter(X_train, y_train, 40, 'g', 'o', alpha=0.8, label='train')
plt.scatter(X_test, y_test, 40, 'b', 'x', alpha=0.8, label='test')
plt.plot(X_train, y_best_predict_train, 'c', label='Полином ' + str(best_degree) + ' степени без регуляризации')
plt.plot(X_train, y6_predict_train, 'y', label='Полином 6 степени без регуляризации')
plt.plot(X_train, y6_a_predict_train, 'r', label='Полином 6 степени с регуляризацией')
plt.legend(loc = 'best', prop = {'size': 10})
plt.title('Полиномиальная регрессия')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### Задание 3. Линейная регрессия в scikit-learn

Скачайте файлы ``flats_moscow.txt`` и ``flats_moscow_description.pdf``. В первом из них содержатся данные о квартирах в Москве. Каждая строка содержит шесть характеристик некоторой квартиры, разделённые знаками табуляции; в первой строке записаны кодовые названия характеристик. Во втором файле приведены краткие описания признаков. Вашей задачей будет построить с помощью линейной регрессии зависимость между ценой квартиры и остальными доступными параметрами.

Построив несколько моделей, нужно сравнить их качество. Проверять это на той же выборке, на которой вы обучались, бессмысленно и даже вредно (вспомните пример с полиномами: как хорошо падала ошибка на обучающей выборке с ростом степени и как быстро росла ошибка на тестовых данных!). Поэтому вам нужно будет разделить выборку на обучающую и тестовую. Делать это лучше случайным образом (ведь вы не знаете, как создатели набора данных упорядочили объекты); рекомендуем вам для этого функцию `sklearn.model_selection.train_test_split`.

Постройте следующие модели линейной регрессии по методу наименьших квадратов:

1. на основе собственной функции (нормальное уравнение);

1. с L2-регуляризацией на основе собственной функции – параметр регуляризации нужно подобрать;

1. собственную реализацию полиномиальной регрессии – степень полинома нужно подобрать;

1. на основе функции `LinearRegression` из `scikit-learn`;

1. с L2-регуляризацией на основе функции `Ridge` из `scikit-learn` – параметр регуляризации нужно подобрать;

1. с L1-регуляризацией на основе функции `Lasso` из `scikit-learn` – параметр регуляризации нужно подобрать.

Выведите и сравните регрессионные коэффициенты для всех функций. Какой смысл имеют их знаки? Согласуются ли они с вашими представлениями о жизни?

Оцените качество решения задачи, выведя среднеквадратическую ошибку на обучающих и тестовых данных для всех функций. Эти ошибки лучше свести в таблицу (например, используя `pandas.DataFrame`).

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('flats_moscow.txt', sep='\t')
print(df.head())

In [ ]:
X = df.drop(columns=['price']) # В признаки занесем все без порядкового номера и стоимости
y = df['price'] # В значения занесем стоимость
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, random_state=42) # Случайное разделение на train и test в отношении 2:1 

In [ ]:
# Функция нахождения значений по линейной функции с предсказанными весовыми коэффициентами
def f_reg(X, w):
    X = np.column_stack([np.ones(X.shape[0]).T, X])
    y_predict = np.dot(w, X.T)
    return y_predict

In [ ]:
def f_err(X_train, y_train, X_test, y_test, w, df_mse, i, s):
    y_predict_train = f_reg(X_train, w)
    mse_train = mean_squared_error(y_train, y_predict_train)
    y_predict_test = f_reg(X_test, w)
    mse_test = mean_squared_error(y_test, y_predict_test)
    df_mse.loc[i] = [s, mse_train, mse_test]
    return df_mse

In [ ]:
# Полиномиальная регрессия на основе нормального уравнения
def poli_reg1(X_train, X_test, y_train, degree):
    poly = PolynomialFeatures(degree, include_bias=False)
    X_poly_train = poly.fit_transform(X_train)
    X_poly_test = poly.fit_transform(X_test)    
    w_0 = np.ones(X_poly_train.shape[1] + 1)
    w = get_weight(X_poly_train, y_train)
    return X_poly_train, X_poly_test, w

In [ ]:
i = 0

df_mse = pd.DataFrame({'model': [], 'mse_train': [], 'mse_test': []}) # Датафрейм для сравнения качества решения задачи

print('\n1. На основе собственной функции (нормальное уравнение)')
w = get_weight(X_train, y_train)
print('w = ', w)
i += 1
df_mse = f_err(X_train, y_train, X_test, y_test, w, df_mse, i, 'На основе собственной функции (нормальное уравнение)')

print('\n2. L2-регуляризация на основе собственной функции')
alpha_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
mse_train = []
mse_test = []
arr_w = []
for a in alpha_values:
    w = get_weight(X_train, y_train, a)
    arr_w.append(w)
    y_predict_train = f_reg(X_train, w)
    mse_train.append(mean_squared_error(y_train, y_predict_train))
    y_predict_test = f_reg(X_test, w)
    mse_test.append(mean_squared_error(y_test, y_predict_test))
w = arr_w[mse_test.index(min(mse_test))]
alpha = alpha_values[mse_test.index(min(mse_test))]
print('alpha =', alpha, '\tw = ', w)
i += 1
df_mse = f_err(X_train, y_train, X_test, y_test, w, df_mse, i, 'L2-регуляризация на основе собственной функции')

print('\n3. Cобственная реализация полиномиальной регрессии')
X_tr = np.array(X_train)
X_te = np.array(X_test)
y_tr = np.array(y_train)
y_te = np.array(y_test)
count = 10
min_degree = 1
best_mse_train = 1e+10
best_mse_test = 1e+10
best_w = []
best_X_train = []
best_X_test = []

for d in range(count):
    X_poly_train, X_poly_test, w = poli_reg1(np.array(X_train), np.array(X_test), np.array(y_train), d + min_degree)
    
    y_predict_train = f_reg(X_poly_train, w)
    y_predict_test = f_reg(X_poly_test, w)
    
    mse_tr = mean_squared_error(y_tr, y_predict_train)
    #r2_train = r2_score(y_tr, y_predict_train)
            
    mse_te = mean_squared_error(y_te, y_predict_test)
    # r2_test = r2_score(y_te, y_predict_test))
    
    if mse_te < best_mse_test:
        best_mse_train = mse_tr
        best_mse_test = mse_te
        best_w = w
        best_X_train = X_poly_train
        best_X_test = X_poly_test
        best_degree = d+min_degree
print('degree =', best_degree, '\tw = ', best_w)
i += 1
df_mse = f_err(best_X_train, y_tr, best_X_test, y_te, best_w, df_mse, i, 'Cобственная реализация полиномиальной регрессии')

print('\n4. На основе функции LinearRegression')
# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
w = np.concatenate([[model.intercept_], model.coef_])
print('w = ', w)
i += 1
df_mse = f_err(X_train, y_train, X_test, y_test, w, df_mse, i, 'На основе функции LinearRegression')

print('\n5. C L2-регуляризацией на основе функции Ridge')
best_mse = 1e+10
best_w = []
alpha_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
for alpha in alpha_values:
    # Обучение модели
    model = Ridge(alpha)
    model.fit(X_train, y_train)
    # Оценка модели
    w = np.concatenate([[model.intercept_], model.coef_])
    y_predict = f_reg(X_test, w)
    mse = mean_squared_error(y_test, y_predict)
    if mse < best_mse:
        best_mse = mse
        best_w = w
        best_alpha = alpha
print('alpha =', best_alpha,'\tw = ', best_w)
i += 1
df_mse = f_err(X_train, y_train, X_test, y_test, w, df_mse, i, 'C L2-регуляризацией на основе функции Ridge')

print('\n6. C L1-регуляризацией на основе функции Lasso')
best_mse = 1e+10
best_w = []
alpha_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
for alpha in alpha_values:
    # Обучение модели
    model = Lasso(alpha)
    model.fit(X_train, y_train)
    # Оценка модели
    w = np.concatenate([[model.intercept_], model.coef_])
    y_predict = f_reg(X_test, w)
    mse = mean_squared_error(y_test, y_predict)
    if mse < best_mse:
        best_mse = mse
        best_w = w
        best_alpha = alpha
print('alpha =', best_alpha,'\tw = ', best_w)
i += 1
df_mse = f_err(X_train, y_train, X_test, y_test, w, df_mse, i, 'C L1-регуляризацией на основе функции Lasso')

# Вывод ошибок на обучающем и тестовом наборах для построенных функций
print('\nСреднеквадратичные ошибки на обучающем и тестовом наборах для построенных функций\n')
print(df_mse)

>_Вывод: Знаки "-" у коэффициентов при dist и metrdist говорят о том, что эти параметры отрицательно сказываются на цене (чем больше расстояние, тем ниже цена). 

Конечно, никто не гарантирует, что объясняемая переменная (цена квартиры) зависит от остальных характеристик именно линейно. Зависимость может быть, например, квадратичной или логарифмической; больше того, могут быть важны не только отдельные признаки, но и их комбинации. Это можно учитывать, добавляя в качестве дополнительных признаков разные функции от уже имеющихся характеристик: их квадраты, логарифмы, попарные произведения.

В этом задании вам нужно постараться улучшить качество модели, добавляя дополнительные признаки (не менее трёх), являющиеся функциями от уже имеющихся. Но будьте осторожны: чрезмерное усложнение модели будет приводить к переобучению.

In [ ]:
# Определение функции ошибки
def get_mse(f, X_train, y_train, X_test, y_test):
    # Добавляем значения выбранной функции для признаков
    f_X_train = f(X_train)
    new_X_train = np.concatenate((X_train, f_X_train), axis = 1)
    f_X_test = f(X_test)
    new_X_test = np.concatenate((X_test, f_X_test), axis = 1)
    # Нахождение коэффициентов регрессии на основе нормального уравнения
    w = get_weight(new_X_train, y_train)
    # Нахождение предсказанных значений и функции ошибки 
    y_predict_train = f_reg(new_X_train, w)
    mse_train = mean_squared_error(y_train, y_predict_train)
    y_predict_test = f_reg(new_X_test, w)
    mse_test = mean_squared_error(y_test, y_predict_test)
    return mse_train, mse_test

# Функция добавления произведений пар признаков
def pair_X(X):
    X = np.array(X)
    new_X = np.array([]).reshape(X.shape[0], 0)
    for i in range(X.shape[1] - 1):
        for j in range(i, X.shape[1]):
            new_column = np.multiply(X[:,i], X[:,j])
            new_X = np.column_stack((new_X, new_column))
    return new_X

add_funcs = {
    'none': lambda X: np.array([]).reshape(X.shape[0], 0),
    'square': lambda X : X ** 2,
    'cube': lambda X : X ** 3,
    'fourth': lambda X : X ** 4,
    'fifth': lambda X : X ** 5,
    'six': lambda X : X ** 6,
    'seven': lambda X : X ** 7,
    'eight': lambda X : X ** 8,
    'nine': lambda X : X ** 9,
    'tenth': lambda X : X ** 10,
    'log': lambda X : np.log(X),
    'xlog': lambda X : X * np.log(X),
    'pair_X': pair_X
}

min_mse_test = 1e+10
best_func = ''
for name, f in add_funcs.items():
    mse_train, mse_test = get_mse(f, X_train, y_train, X_test, y_test)
    if mse_test < min_mse_test:
        min_mse_test = mse_test
        best_func = name
    print('func:',name,'\tmse_train =', mse_train, '\tmse_test =', mse_test)
print('\nBest func:', best_func)